<a href="https://colab.research.google.com/github/Ishfak46/Customer-Segmention-DA/blob/main/Customer_Segmention_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data wrangling
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# for data preprocessing and clustering
from sklearn.cluster import KMeans

%matplotlib inline
# to include graphs inline within the frontends next to code

%config InlineBackend.figure_format='retina'
#to enable retina (high resolution) plots

pd.options.mode.chained_assignment = None
# to bypass warnings in various dataframe assignments

In [ ]:
# load data into a dataframe
customers_orders = pd.read_csv("Orders - Analysis Task.csv")

In [ ]:
# first rows of the dataset
customers_orders.head()


,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
0,DPR,DPR,100,AD-982-708-895-F-6C894FB,52039657.0,1312378.0,8.329072e+13,04/12/2018,2.0,200.0,-200.00,0.00,0.0,0.0,0.0,0.0,2.0
1,RJF,Product P,28 / A / MTM,83-490-E49-8C8-8-3B100BC,56914686.0,3715657.0,3.625379e+13,01/04/2019,2.0,190.0,-190.00,0.00,0.0,0.0,0.0,0.0,2.0
2,CLH,Product B,32 / B / FtO,68-ECA-BC7-3B2-A-E73DE1B,24064862.0,9533448.0,7.309456e+13,05/11/2018,0.0,164.8,-156.56,-8.24,0.0,0.0,0.0,-2.0,2.0
3,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868.0,4121004.0,5.361658e+13,19/02/2019,1.0,119.0,-119.00,0.00,0.0,0.0,0.0,0.0,1.0
4,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868.0,4121004.0,2.926322e+13,19/02/2019,1.0,119.0,-119.00,0.00,0.0,0.0,0.0,0.0,1.0


In [ ]:
# first glance of customers_orders data
customers_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47345 entries, 0 to 47344
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_title           47345 non-null  object 
 1   product_type            47345 non-null  object 
 2   variant_title           47345 non-null  object 
 3   variant_sku             47345 non-null  object 
 4   variant_id              47344 non-null  float64
 5   customer_id             47344 non-null  float64
 6   order_id                47344 non-null  float64
 7   day                     47344 non-null  object 
 8   net_quantity            47344 non-null  float64
 9   gross_sales             47344 non-null  float64
 10  discounts               47344 non-null  float64
 11  returns                 47344 non-null  float64
 12  net_sales               47344 non-null  float64
 13  taxes                   47344 non-null  float64
 14  total_sales             47344 non-null

In [ ]:
# descriptive statistics of the non-object columns
customers_orders.describe()

,variant_id,customer_id,order_id,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
count,4.734400e+04,4.734400e+04,4.734400e+04,47344.000000,47344.000000,47344.000000,47344.000000,47344.000000,47344.000000,47344.000000,47344.000000,47344.000000
mean,2.901852e+11,5.651417e+11,5.516670e+13,1.013412,79.178062,-7.324068,-0.022001,71.831993,14.083969,85.915962,-0.000486,1.013898
std,4.627216e+12,6.085614e+12,2.587541e+13,0.131646,15.533390,8.481811,1.272101,17.537304,3.799683,20.654603,0.023876,0.130062
min,1.001447e+07,1.000661e+06,1.000657e+13,0.000000,24.160000,-200.000000,-142.500000,0.000000,0.000000,0.000000,-2.000000,1.000000
25%,2.692223e+07,3.307570e+06,3.285882e+13,1.000000,74.170000,-14.840000,0.000000,63.040000,12.330000,75.000000,0.000000,1.000000
50%,4.494514e+07,5.572352e+06,5.526232e+13,1.000000,79.170000,-5.200000,0.000000,74.160000,14.830000,89.000000,0.000000,1.000000
75%,7.484197e+07,7.808716e+06,7.744459e+13,1.000000,79.180000,0.000000,0.000000,79.170000,15.830000,95.000000,0.000000,1.000000
max,8.422212e+13,9.954992e+13,9.999554e+13,6.000000,445.000000,0.000000,0.000000,445.000000,63.340000,445.000000,0.000000,6.000000


In [ ]:
print("Number of rows that net quantity is negative:",
      customers_orders[customers_orders.net_quantity < 0].shape[0])

Number of rows that net quantity is negative: 0


In [ ]:
# exclude not sold/ordered SKUs from the dataset
customers_orders = customers_orders[
    customers_orders["ordered_item_quantity"] > 0]

In [ ]:
def encode_column(column):
    if column > 0:
        return 1
    if column <= 0:
        return 0


def aggregate_by_ordered_quantity(dataframe, column_list):
    '''this function:
    1. aggregates a given dataframe by column list,
    as a result creates a aggregated dataframe by counting the ordered item quantities

    2. adds number_of_X ordered where X is the second element in the column_list
    to the aggregated dataframe by encoding ordered items into 1

    3. creates final dataframe containing information about
    how many of X are ordered, based on the first element passed in the column list'''

    aggregated_dataframe = (dataframe
                            .groupby(column_list)
                            .ordered_item_quantity.count()
                            .reset_index())

    aggregated_dataframe["products_ordered"] = (aggregated_dataframe
                                                 .ordered_item_quantity
                                                 .apply(encode_column))

    final_dataframe = (aggregated_dataframe
                       .groupby(column_list[0])
                       .products_ordered.sum() # aligned with the added column name
                       .reset_index())

    return final_dataframe

In [ ]:
# apply functions to customers_orders
customers = aggregate_by_ordered_quantity(customers_orders, ["customer_id", "product_type"])


In [ ]:
print(customers.head())

   customer_id  products_ordered
0    1000661.0                 1
1    1001914.0                 1
2    1002167.0                 2
3    1002387.0                 1
4    1002419.0                 2


In [ ]:
# aggregate data per customer_id and order_id,
# to see ordered item sum and returned item sum
ordered_sum_by_customer_order = (customers_orders
                                 .groupby(["customer_id", "order_id"])
                                 .ordered_item_quantity.sum()
                                 .reset_index())

returned_sum_by_customer_order = (customers_orders
                                  .groupby(["customer_id", "order_id"])
                                  .returned_item_quantity.sum()
                                  .reset_index())

# merge two dataframes to be able to calculate unit return rate
ordered_returned_sums = pd.merge(ordered_sum_by_customer_order, returned_sum_by_customer_order)


In [ ]:
# calculate unit return rate per order and customer
ordered_returned_sums["average_return_rate"] = (-1 *
                                             ordered_returned_sums["returned_item_quantity"] /
                                             ordered_returned_sums["ordered_item_quantity"])


In [ ]:
ordered_returned_sums.head()

,customer_id,order_id,ordered_item_quantity,returned_item_quantity,average_return_rate
0,1000661.0,9.911999e+13,3.0,0.0,-0.0
1,1001914.0,7.975857e+13,1.0,0.0,-0.0
2,1002167.0,5.744015e+13,1.0,0.0,-0.0
3,1002167.0,5.882552e+13,1.0,0.0,-0.0
4,1002387.0,8.427433e+13,1.0,0.0,-0.0


In [ ]:
# take average of the unit return rate for all orders of a customer
customer_return_rate = (ordered_returned_sums
                        .groupby("customer_id")
                        .average_return_rate
                        .mean()
                        .reset_index())

In [ ]:
return_rates = pd.DataFrame(customer_return_rate["average_return_rate"]
                            .value_counts()
                            .reset_index())

return_rates.rename(columns=
                    {"index": "average return rate",
                     "average_return_rate": "count of unit return rate"},
                    inplace=True)

return_rates.sort_values(by="average return rate")

,average return rate,count of unit return rate
0,0.000000,22179
5,0.083333,1
4,0.250000,1
2,0.333333,4
3,0.500000,3
6,0.666667,1
1,1.000000,5


In [ ]:
# add average_return_rate to customers dataframe
customers = pd.merge(customers,
                     customer_return_rate,
                     on="customer_id")


In [ ]:
# aggreagate total sales per customer id
customer_total_spending = (customers_orders
                           .groupby("customer_id")
                           .total_sales
                           .sum()
                           .reset_index())

customer_total_spending.rename(columns = {"total_sales" : "total_spending"},
                               inplace = True)

In [ ]:
# add total sales to customers dataframe
customers = customers.merge(customer_total_spending,
                            on="customer_id")

In [ ]:
print("The number of customers from the existing customer base:", customers.shape[0])


The number of customers from the existing customer base: 22194


In [ ]:
# drop id column since it is not a feature
customers.drop(columns="customer_id",
               inplace=True)

In [ ]:

customers.head()

,products_ordered,average_return_rate,total_spending
0,1,0.0,260.0
1,1,0.0,79.2
2,2,0.0,168.2
3,1,0.0,89.0
4,2,0.0,103.0


In [ ]:
fig = make_subplots(rows=3, cols=1,
                   subplot_titles=("Products Ordered",
                                   "Average Return Rate",
                                   "Total Spending"))

fig.append_trace(go.Histogram(x=customers.products_ordered),
                 row=1, col=1)

fig.append_trace(go.Histogram(x=customers.average_return_rate),
                 row=2, col=1)

fig.append_trace(go.Histogram(x=customers.total_spending),
                 row=3, col=1)

fig.update_layout(height=800, width=800,
                  title_text="Distribution of the Features")

fig.show()

In [ ]:
def apply_log1p_transformation(dataframe, column):
    '''This function takes a dataframe and a column in the string format
    then applies numpy log1p transformation to the column
    as a result returns log1p applied pandas series'''

    dataframe["log_" + column] = np.log1p(dataframe[column])
    return dataframe["log_" + column]

In [ ]:
apply_log1p_transformation(customers, "products_ordered")

0        0.693147
1        0.693147
2        1.098612
3        0.693147
4        1.098612
           ...   
22189    0.693147
22190    0.693147
22191    0.693147
22192    1.098612
22193    0.693147
Name: log_products_ordered, Length: 22194, dtype: float64

In [ ]:
apply_log1p_transformation(customers, "average_return_rate")

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
22189    0.0
22190    0.0
22191    0.0
22192    0.0
22193    0.0
Name: log_average_return_rate, Length: 22194, dtype: float64

In [ ]:
apply_log1p_transformation(customers, "total_spending")

0        5.564520
1        4.384524
2        5.131081
3        4.499810
4        4.644391
           ...   
22189    4.787492
22190    5.201256
22191    4.499810
22192    5.590987
22193    4.174387
Name: log_total_spending, Length: 22194, dtype: float64

In [ ]:
fig = make_subplots(rows=3, cols=1,
                   subplot_titles=("Products Ordered",
                                   "Average Return Rate",
                                   "Total Spending"))

fig.append_trace(go.Histogram(x=customers.log_products_ordered),
                 row=1, col=1)

fig.append_trace(go.Histogram(x=customers.log_average_return_rate),
                 row=2, col=1)

fig.append_trace(go.Histogram(x=customers.log_total_spending),
                 row=3, col=1)

fig.update_layout(height=800, width=800,
                  title_text="Distribution of the Features after Logarithm Transformation")

fig.show()


In [ ]:
customers.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending
0,1,0.0,260.0,0.693147,0.0,5.564520
1,1,0.0,79.2,0.693147,0.0,4.384524
2,2,0.0,168.2,1.098612,0.0,5.131081
3,1,0.0,89.0,0.693147,0.0,4.499810
4,2,0.0,103.0,1.098612,0.0,4.644391


In [ ]:
# features we are going to use as an input to the model
customers.iloc[:,3:]

,log_products_ordered,log_average_return_rate,log_total_spending
0,0.693147,0.0,5.564520
1,0.693147,0.0,4.384524
2,1.098612,0.0,5.131081
3,0.693147,0.0,4.499810
4,1.098612,0.0,4.644391
...,...,...,...
22189,0.693147,0.0,4.787492
22190,0.693147,0.0,5.201256
22191,0.693147,0.0,4.499810
22192,1.098612,0.0,5.590987


In [ ]:
# create initial K-means model
kmeans_model = KMeans(init='k-means++',
                      max_iter=500,
                      random_state=42)

In [ ]:
kmeans_model.fit(customers.iloc[:,3:])

# print the sum of distances from all examples to the center of the cluster
print("within-cluster sum-of-squares (inertia) of the model is:", kmeans_model.inertia_)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



within-cluster sum-of-squares (inertia) of the model is: 721.1200127099326


In [ ]:
def make_list_of_K(K, dataframe):
    '''inputs: K as integer and dataframe
    apply k-means clustering to dataframe
    and make a list of inertia values against 1 to K (inclusive)
    return the inertia values list
    '''

    cluster_values = list(range(1, K+1))
    inertia_values=[]

    for c in cluster_values:
        model = KMeans(
            n_clusters = c,
            init='k-means++',
            max_iter=500,
            random_state=42)
        model.fit(dataframe)
        inertia_values.append(model.inertia_)

    return inertia_values

In [ ]:
# save inertia values in a dataframe for k values between 1 to 15
results = make_list_of_K(15, customers.iloc[:, 3:])

k_values_distances = pd.DataFrame({"clusters": list(range(1, 16)),
                                   "within cluster sum of squared distances": results})

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

In [ ]:
# visualization for the selection of number of segments
fig = go.Figure()

fig.add_trace(go.Scatter(x=k_values_distances["clusters"],
                         y=k_values_distances["within cluster sum of squared distances"],
                         mode='lines+markers'))

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1),
                  title_text="Within Cluster Sum of Squared Distances VS K Values",
                  xaxis_title="K values",
                  yaxis_title="Cluster sum of squared distances")

fig.show()

In [ ]:
# create clustering model with optimal k=4
updated_kmeans_model = KMeans(n_clusters = 4,
                              init='k-means++',
                              max_iter=500,
                              random_state=42)

updated_kmeans_model.fit_predict(customers.iloc[:,3:])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([2, 0, 2, ..., 0, 1, 0], dtype=int32)

In [ ]:
# create cluster centers and actual data arrays
cluster_centers = updated_kmeans_model.cluster_centers_
actual_data = np.expm1(cluster_centers)
add_points = np.append(actual_data, cluster_centers, axis=1)
add_points

array([[1.00672257e+00, 3.75288513e-04, 8.34133754e+01, 6.96502828e-01,
        3.75218110e-04, 4.43572587e+00],
       [2.29586799e+00, 1.35219735e-04, 2.95024892e+02, 1.19266956e+00,
        1.35210594e-04, 5.69044354e+00],
       [1.51288963e+00, 3.30649492e-04, 1.72192113e+02, 9.21433338e-01,
        3.30594840e-04, 5.15440146e+00],
       [3.55519393e+00, 5.23853431e-05, 5.78347386e+02, 1.51626811e+00,
        5.23839710e-05, 6.36190227e+00]])

In [ ]:
# add labels to customers dataframe and add_points array
add_points = np.append(add_points, [[0], [1], [2], [3]], axis=1)
customers["clusters"] = updated_kmeans_model.labels_

In [ ]:
# create centers dataframe from add_points
centers_df = pd.DataFrame(data=add_points, columns=["products_ordered",
                                                    "average_return_rate",
                                                    "total_spending",
                                                    "log_products_ordered",
                                                    "log_average_return_rate",
                                                    "log_total_spending",
                                                    "clusters"])
centers_df.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters
0,1.006723,0.000375,83.413375,0.696503,0.000375,4.435726,0.0
1,2.295868,0.000135,295.024892,1.192670,0.000135,5.690444,1.0
2,1.512890,0.000331,172.192113,0.921433,0.000331,5.154401,2.0
3,3.555194,0.000052,578.347386,1.516268,0.000052,6.361902,3.0


In [ ]:
# align cluster centers of centers_df and customers
centers_df["clusters"] = centers_df["clusters"].astype("int")

In [ ]:
centers_df.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters
0,1.006723,0.000375,83.413375,0.696503,0.000375,4.435726,0
1,2.295868,0.000135,295.024892,1.192670,0.000135,5.690444,1
2,1.512890,0.000331,172.192113,0.921433,0.000331,5.154401,2
3,3.555194,0.000052,578.347386,1.516268,0.000052,6.361902,3


In [ ]:
customers.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters
0,1,0.0,260.0,0.693147,0.0,5.564520,2
1,1,0.0,79.2,0.693147,0.0,4.384524,0
2,2,0.0,168.2,1.098612,0.0,5.131081,2
3,1,0.0,89.0,0.693147,0.0,4.499810,0
4,2,0.0,103.0,1.098612,0.0,4.644391,0


In [ ]:
# differentiate between data points and cluster centers
customers["is_center"] = 0
centers_df["is_center"] = 1

# add dataframes together
customers = customers.append(centers_df, ignore_index=True)

<ipython-input-45-1dcf0956e607>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
customers.tail()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters,is_center
22193,1.000000,0.000000,64.000000,0.693147,0.000000,4.174387,0,0
22194,1.006723,0.000375,83.413375,0.696503,0.000375,4.435726,0,1
22195,2.295868,0.000135,295.024892,1.192670,0.000135,5.690444,1,1
22196,1.512890,0.000331,172.192113,0.921433,0.000331,5.154401,2,1
22197,3.555194,0.000052,578.347386,1.516268,0.000052,6.361902,3,1


In [ ]:
# add clusters to the dataframe
customers["cluster_name"] = customers["clusters"].astype(str)

In [ ]:
# visualize log_transformation customer segments with a 3D plot
fig = px.scatter_3d(customers,
                    x="log_products_ordered",
                    y="log_average_return_rate",
                    z="log_total_spending",
                    color='cluster_name',
                    hover_data=["products_ordered",
                                "average_return_rate",
                                "total_spending"],
                    category_orders = {"cluster_name":
                                       ["0", "1", "2", "3"]},
                    symbol = "is_center"
                    )

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
# values for log_transformation
cardinality_df = pd.DataFrame(
    customers.cluster_name.value_counts().reset_index())

cardinality_df.rename(columns={"index": "Customer Groups",
                               "cluster_name": "Customer Group Magnitude"},
                      inplace=True)

In [ ]:
cardinality_df

,Customer Groups,Customer Group Magnitude
0,0,10599
1,2,6291
2,1,3786
3,3,1522


In [ ]:
fig = px.bar(cardinality_df, x="Customer Groups",
             y="Customer Group Magnitude",
             color = "Customer Groups",
             category_orders = {"Customer Groups": ["0", "1", "2", "3"]})

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1),
                 yaxis = dict(
        tickmode = 'linear',
        tick0 = 1000,
        dtick = 1000))

fig.show()